In [1]:
# Import modules
import pandas as pd
import json
import requests
import pprint
from config import token
from config import userId

# Career One Stop API Calls

In [2]:
# Configure Header
headers = {'Authorization': f'Bearer {token}'}

## Step 1: Get list of occupations

In [3]:
keyword = '15-'
dataLevelOnly = 'N'
startRecord = 0
limitRecord = 1000

api_url = f'https://api.careeronestop.org/v1/occupation/{userId}/{keyword}/{dataLevelOnly}/{startRecord}/{limitRecord}?'
response1 = requests.get(api_url, headers=headers).json()

In [4]:
# occupation_df = pd.DataFrame()
# occupation_ls = []
# for x in response1['OccupationList']:
#      occupation_ls.append(x['OnetCode'])
# occupation_ls

## Step 2: Get Occupation Wages & Education

In [5]:
occupation_df = pd.DataFrame()
location = 'FL'

for x in response1['OccupationList']:
    # API call to retrieve occupation details
    keyword = x['OnetCode']
    params = {'wages': 'true'
             ,'training': 'true'
             }
    api_url = f'https://api.careeronestop.org/v1/occupation/{userId}/{keyword}/{location}?'
    response2 = requests.get(api_url, headers=headers, params=params).json()
    
    # Loop through wage rates & assign the annual & hourly valiables accordingly
    annual = 0
    hourly = 0
    for row in response2['OccupationDetail'][0]['Wages']['StateWagesList']:
        if row['RateType'] == 'Annual':
            annual = row['Median']
        elif row['RateType'] == 'Hourly':
            hourly = row['Median']
    
    # Loop through education levels & assign valiables accordingly
    EducationLevel_1 = 0
    EducationLevel_2 = 0
    EducationLevel_3 = 0
    EducationLevel_4 = 0
    EducationLevel_5 = 0
    EducationLevel_6 = 0
    EducationLevel_7 = 0

    for row in response2['OccupationDetail'][0]['EducationTraining']['EducationType']:
        if row['EducationLevel'] == 'Less than high school diploma':
            EducationLevel_1 = row['Value']
        elif row['EducationLevel'] == 'High school diploma or equivalent':
            EducationLevel_2 = row['Value']
        elif row['EducationLevel'] == 'Some college, no degree':
            EducationLevel_3 = row['Value']
        elif row['EducationLevel'] == "Associate's degree":
            EducationLevel_4 = row['Value']
        elif row['EducationLevel'] == "Bachelor's degree":
            EducationLevel_5 = row['Value']
        elif row['EducationLevel'] == "Master's degree":
            EducationLevel_6 = row['Value']
        elif row['EducationLevel'] == 'Doctoral or professional degree':
            EducationLevel_7 = row['Value']
            
    # Build DataFrame with this instance
    df = pd.DataFrame(
            {'OnetCode':[response2['OccupationDetail'][0]['OnetCode']]
            ,'OnetTitle':[response2['OccupationDetail'][0]['OnetTitle']]
            ,'OnetDescription':[response2['OccupationDetail'][0]['OnetDescription']]
            ,'Annual':[annual]
            ,'Hourly':[hourly]
            ,'WageYear':[response2['OccupationDetail'][0]['Wages']['WageYear']]
            ,'BrightOutlook':[response2['OccupationDetail'][0]['BrightOutlook']]
            ,'EducationLevel_1':[EducationLevel_1]
            ,'EducationLevel_2':[EducationLevel_2]
            ,'EducationLevel_3':[EducationLevel_3]
            ,'EducationLevel_4':[EducationLevel_4]
            ,'EducationLevel_5':[EducationLevel_5]
            ,'EducationLevel_6':[EducationLevel_6]
            ,'EducationLevel_7':[EducationLevel_7]
            ,'EducationCode':[response2['OccupationDetail'][0]['EducationTraining']['EducationCode']]
            ,'EducationTitle':[response2['OccupationDetail'][0]['EducationTraining']['EducationTitle']]
            ,'ExperienceCode':[response2['OccupationDetail'][0]['EducationTraining']['ExperienceCode']]
            ,'ExperienceTitle':[response2['OccupationDetail'][0]['EducationTraining']['ExperienceTitle']]
            ,'TrainingCode':[response2['OccupationDetail'][0]['EducationTraining']['TrainingCode']]
            ,'TrainingTitle':[response2['OccupationDetail'][0]['EducationTraining']['TrainingTitle']]
            ,'OccupationTitle':[response2['OccupationDetail'][0]['EducationTraining']['OccupationTitle']]
            })
    occupation_df = occupation_df.append(df)

occupation_df = occupation_df.set_index('OnetCode')
occupation_df.head(5)

,OnetTitle,OnetDescription,Annual,Hourly,WageYear,BrightOutlook,EducationLevel_1,EducationLevel_2,EducationLevel_3,EducationLevel_4,EducationLevel_5,EducationLevel_6,EducationLevel_7,EducationCode,EducationTitle,ExperienceCode,ExperienceTitle,TrainingCode,TrainingTitle,OccupationTitle
OnetCode,,,,,,,,,,,,,,,,,,,,
15-1255.01,Video Game Designers,Design core features of video games. Specify i...,63110,30.34,2019,Bright,.7,4.9,15.4,10.4,53.7,13.6,1.3,4,Associate's degree,3,No work experience,6,No on-the-job training,Web Developers and Digital Interface Designers
15-1243.00,Database Architects,"Design strategies for enterprise databases, da...",90230,43.38,2019,Bright,.5,4.5,12,8.2,46.9,25.2,2.7,3,Bachelor's degree,3,No work experience,6,No on-the-job training,Database Administrators and Architects
15-1252.00,Software Developers,"Research, design, and develop computer and net...",95930,46.12,2019,Bright,.4,2.2,7.4,3.9,51.5,30.3,4.3,3,Bachelor's degree,3,No work experience,6,No on-the-job training,Software Developers and Software Quality Assur...
15-1243.01,Data Warehousing Specialists,"Design, model, or implement corporate data war...",90230,43.38,2019,Bright,.5,4.5,12,8.2,46.9,25.2,2.7,3,Bachelor's degree,3,No work experience,6,No on-the-job training,Database Administrators and Architects
15-1255.00,Web and Digital Interface Designers,Design digital user interfaces or websites. De...,63110,30.34,2019,Bright,.7,4.9,15.4,10.4,53.7,13.6,1.3,4,Associate's degree,3,No work experience,6,No on-the-job training,Web Developers and Digital Interface Designers


## Step 3: Print to csv

In [6]:
# Write dataframe to CSV, without the Pandas index
occupation_df.to_csv("Clean_Data/CareerOneStop.csv", index=False, header=True)